In [61]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings;
warnings.filterwarnings("ignore")

In [62]:
df=pd.read_csv("C:\\Users\Shivansh\Downloads\cardio_train.csv",sep=";")
df.isnull().sum()
df.dtypes


id               int64
age              int64
gender           int64
height           int64
weight         float64
ap_hi            int64
ap_lo            int64
cholesterol      int64
gluc             int64
smoke            int64
alco             int64
active           int64
cardio           int64
dtype: object

In [63]:
df.head(5)
df.drop("id",inplace=True,axis=1)


In [64]:
df.head(5)
arr=np.array(df["age"])
ans=[]
for val in arr:
    ans.append(int (val/365))
df["Age"]=pd.DataFrame(ans)
df.drop("age",inplace=True,axis=1)
df.drop("alco",inplace=True,axis=1)
df.head(5)

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,active,cardio,Age
0,2,168,62.0,110,80,1,1,0,1,0,50
1,1,156,85.0,140,90,3,1,0,1,1,55
2,1,165,64.0,130,70,3,1,0,0,1,51
3,2,169,82.0,150,100,1,1,0,1,1,48
4,1,156,56.0,100,60,1,1,0,0,0,47


In [65]:
df.head(4)
df["height"]=df["height"]/100


In [66]:
df["BMI"]=df["weight"]/(df["height"]*df["height"])
df.drop("height",axis=1,inplace=True)
df.drop("weight",inplace=True,axis=1)

In [67]:
df["Arterial pressure"]=(df["ap_hi"]+2*df["ap_lo"])/3
df.drop("ap_lo",inplace=True,axis=1)
df.drop("ap_hi",axis=1,inplace=True)

In [78]:
df.columns=["Gender","Cholesterol","Glucose","Smoking","Physical Activity","Cardiovascular Disease","Age","BMI","Arterial pressure"]
df.head(10)

,Gender,Cholesterol,Glucose,Smoking,Physical Activity,Cardiovascular Disease,Age,BMI,Arterial pressure
0,2,1,1,0,1,0,50,21.967120,90.000000
1,1,3,1,0,1,1,55,34.927679,106.666667
2,1,3,1,0,0,1,51,23.507805,90.000000
3,2,1,1,0,1,1,48,28.710479,116.666667
4,1,1,1,0,0,0,47,23.011177,73.333333
5,1,2,2,0,0,0,60,29.384676,93.333333
6,1,3,1,0,1,0,60,37.729725,96.666667
7,2,3,3,0,1,1,61,29.983588,103.333333
8,1,1,1,0,1,0,48,28.440955,83.333333
9,1,1,1,0,0,0,54,25.282570,76.666667


In [69]:
# now data claning is done
plt.figure(figsize=(16,5))

<Figure size 1600x500 with 0 Axes>

<Figure size 1600x500 with 0 Axes>

In [70]:
df1=df.copy()
x=df1.drop("Arterial pressure",axis=1)

In [72]:

VIR=pd.DataFrame()
VIR["Columns"]=x.columns
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIR["Inflanation Factor"]=[variance_inflation_factor(x.values,i) for i in range(len(x.columns))]
VIR

,Columns,Inflanation Factor
0,Gender,9.059219
1,Cholesterol,6.700546
2,Glucose,6.979867
3,Smoking,1.239944
4,Physical Activity,4.875471
5,Cardiovascular Disease,2.186167
6,Age,28.041738
7,BMI,18.407772


In [74]:
# now applying the grid validation Elastic Net with makes more robust and generalized
x=df.drop("Arterial pressure",axis=1)

y=pd.DataFrame(df["Arterial pressure"])
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1)

In [79]:
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
paramgrid={"alpha":[0.001,0.01,0.1,1,10,100]}
EN=ElasticNet()
grd=GridSearchCV(EN,param_grid=paramgrid,cv=5,scoring='r2',verbose=2)
grd.fit(x_train,y_train)
print(grd.best_estimator_)
print(grd.best_params_)
print(grd.best_score_)
pred=grd.best_estimator_.predict(x_test)
print("The accuracy of the Model is : ",r2_score(y_test,pred))
x_test

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ........................................

,Gender,Cholesterol,Glucose,Smoking,Physical Activity,Cardiovascular Disease,Age,BMI
6670,1,1,1,0,1,1,44,39.728310
49567,1,1,1,0,1,1,39,36.106941
50796,1,1,1,0,1,1,54,24.242424
22310,2,2,2,1,1,1,49,28.409091
54037,2,1,1,0,0,1,46,28.650138
...,...,...,...,...,...,...,...,...
35736,2,1,1,1,0,1,61,26.528510
45283,1,1,1,0,0,0,43,28.604765
1541,1,1,1,0,1,0,45,24.314124
51612,1,1,1,0,1,0,47,22.892820


In [76]:
import joblib

In [77]:
joblib.dump(grd.best_estimator_,"bp_model.pkl")


['bp_model.pkl']